In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from pathlib import Path
from collections import defaultdict
import pytz
import smtplib
import os

ZONE_DEFINITIONS = [
    ("Premium+", float("inf"), "Purple upper"),
    ("Premium", "Purple upper", "Red  Upper"),
    ("Plus+", "Red  Upper", "Yellow Upper"),
    ("Fair", "Yellow Upper", "Green"),
    ("Budget", "Green", "Yellow Lower"),
    ("Discount", "Yellow Lower", "Red Lower"),
    ("Clearance", "Red Lower", "Purple lower"),
    ("Reset", "Purple lower", float("-inf")),
]

# === Current Zone Computation ===
def compute_current_zone(price, zone_definitions, level_dict):
    # Prepare level name → value lookup
    levels = {}
    for zone_name, a, b in zone_definitions:
        if isinstance(a, str):
            levels[a] = level_dict.get(a, None)
        if isinstance(b, str):
            levels[b] = level_dict.get(b, None)
    levels["Purple upper"] = level_dict.get("Purple upper", float("inf"))
    levels["Purple lower"] = level_dict.get("Purple lower", float("-inf"))
    
    for zone_name, upper_bound, lower_bound in zone_definitions:
        # Resolve upper/lower boundaries
        if isinstance(upper_bound, str):
            upper_value = levels.get(upper_bound, float("inf"))
        else:
            upper_value = upper_bound
        if isinstance(lower_bound, str):
            lower_value = levels.get(lower_bound, float("-inf"))
        else:
            lower_value = lower_bound
        
        # Is price in this zone?
        if lower_value < price <= upper_value:
            return zone_name
    
    return "Unknown"


# === CONFIG ===
TICKER_LIST = [
    'USDCAD=X', 'USDGBP=X', 'USDNOK=X', 'USDPLN=X', 'USDAUD=X', 'USDSGD=X',
    'USDJPY=X', 'USDZAR=X', 'USDBRL=X', 'EURUSD=X', 'EURGBP=X', 'EURCHF=X',
    'EURPLN=X', 'EURCZK=X', 'EURNZD=X', 'EURSEK=X', 'EURZAR=X', 'EURSGD=X',
    'GBPNOK=X', 'GBPJPY=X', 'GBPAUD=X', 'GBPCAD=X', 'SEKNOK=X', 'SEKJPY=X',
    'CHFNOK=X', 'CADNOK=X', 'AUDNZD=X', 'AUDJPY=X', 'AUDSEK=X', 'AUDCAD=X',
    'NZDSGD=X', 'NZDCHF=X', 'NZDNOK=X', 'SGDJPY=X', 'SGDHKD=X', 'EURCAD=X',
    'USDCHF=X', 'GBPCHF=X', 'EURNOK=X'
]


KEY_LEVELS_FILE = Path(r"C:\Users\T460\Documents\Quant_trading_research\Data Packs & Scripts\Dev_scripts\FX_1D\FX_1D_KEY.xlsx")
PIP_RANGE = 0.001
LOOKBACK_HOURS = 24
since = datetime.utcnow() - timedelta(hours=LOOKBACK_HOURS)
# === Load Key Levels ===
def load_key_levels(filepath):
    df = pd.read_excel(filepath)
    df.set_index("Ticker", inplace=True)
    return df

# === Check if price touched a key level (with Level Name) ===
def check_proximity(level_dict, high, low):
    matches = []
    for level_name, level_value in level_dict.items():
        if pd.isna(level_value):
            continue
        if (low <= level_value + PIP_RANGE) and (high >= level_value - PIP_RANGE):
            matches.append({
                "Level": round(level_value, 5),
                "Level Name": level_name
            })
    return matches

# === Summarize Touch Events ===
def summarize_touch_events(touches):
    from collections import defaultdict
    stats_dict = defaultdict(lambda: {"count": 0, "last_touch": None})

    for touch in touches:
        key = (touch["Ticker"], touch["Level"], touch["Level Name"])
        stats_dict[key]["count"] += 1
        stats_dict[key]["last_touch"] = touch["Time"]

    rows = []
    for (ticker, level, level_name), stats in stats_dict.items():
        sast_time = stats["last_touch"] + timedelta(hours=2)
        rows.append({
            "Ticker": ticker,
            "Level Name": level_name,
            "Level": level,
            "Touches (24h)": stats["count"],
            "Most Recent Touch (SAST)": sast_time.strftime("%Y-%m-%d %H:%M")
        })

    return pd.DataFrame(rows)

# === Main Alert Generator ===
def generate_alert_report():
    global since
    if 'since' not in globals():
        LOOKBACK_HOURS = 24
        since = datetime.utcnow() - timedelta(hours=LOOKBACK_HOURS)
    
    key_levels_df = load_key_levels(KEY_LEVELS_FILE)
    touches = []

    for ticker in TICKER_LIST:
        print(f"[→] Checking {ticker}...")

        try:
            if datetime.utcnow().weekday() >= 5:
                print(f"[ℹ️] Skipping {ticker} → Weekend")
                continue

            data = yf.download(ticker, start=since.strftime('%Y-%m-%d'), interval="1h", progress=False)
            if data.empty:
                print(f"[⚠️] No data for {ticker} — likely weekend or market closed.")
                continue
            data = data.dropna()

            short = ticker.split("=")[0] + "=X" if "=X" in ticker else ticker
            levels_series = key_levels_df.loc[short].dropna()
            level_dict = dict(levels_series)

            for ts, row in data.iterrows():
                matches = check_proximity(level_dict, row.High.item(), row.Low.item())
                if matches:
                    for match in matches:
                        touches.append({
                            "Ticker": ticker,
                            "Level": match["Level"],
                            "Level Name": match["Level Name"],
                            "Time": ts
                        })

        except Exception as e:
            print(f"[❌] Failed for {ticker}: {e}")

    if not touches:
        print("[✅] No key levels touched in past 24 hours.")
        alert_df = pd.DataFrame()  # empty df if no touches
    else:
        alert_df = summarize_touch_events(touches)
        alert_df = alert_df.sort_values(by=["Ticker", "Level Name"])
        print("[✅] Summary of Key Level Touches:\n")
        print(alert_df.to_string(index=False))

    # Return BOTH alert_df and key_levels_df
    return alert_df, key_levels_df

# === PHASE 1B → Key Level Hit Log with From Zone ===

# === FINAL V2 SAFE PATCH → log_key_level_hits() ===
def log_key_level_hits(alert_df, key_levels_df):
    if alert_df.empty:
        print("[⚠️] No key level hits to log.")
        return
    
    # Load current prices → to compute current zone
    print("[→] Preparing current prices and zones for hits...")
    
    os.makedirs("reports", exist_ok=True)
    
    log_records = []
    for _, row in alert_df.iterrows():
        ticker = row["Ticker"]
        level_name = row["Level Name"]
        level_value = row["Level"]
        touch_time = row["Most Recent Touch (SAST)"]

        try:
            # Try 1h first → fallback to daily
            data = yf.download(ticker, period="1d", interval="1h", progress=False)
            if data.empty:
                print(f"[⚠️] No 1H data for {ticker} → trying Daily...")
                data = yf.download(ticker, period="5d", interval="1d", progress=False)

            if data.empty:
                print(f"[ℹ️] Skipping {ticker} → Market likely closed (no data)")
                continue
                
            data = data.dropna()

            current_price = data["Close"].iloc[-1].item()

            # Prepare level dict
            short = ticker.split("=")[0] + "=X" if "=X" in ticker else ticker
            levels_series = key_levels_df.loc[short].dropna()
            level_dict = dict(levels_series)

            # Current Zone
            current_zone = compute_current_zone(current_price, ZONE_DEFINITIONS, level_dict)

            # From Zone → assume level_value is approximate price at touch
            from_zone = compute_current_zone(level_value, ZONE_DEFINITIONS, level_dict)

            log_records.append({
                "Timestamp": pd.Timestamp.utcnow().strftime("%Y-%m-%d %H:%M:%S"),
                "Ticker": ticker,
                "Level Name": level_name,
                "Level": level_value,
                "Touch Time (UTC)": pd.to_datetime(touch_time).tz_localize('Africa/Johannesburg').tz_convert('UTC').strftime("%Y-%m-%d %H:%M:%S"),
                "From Zone": from_zone,
                "Current Zone": current_zone,
                "Current Price": current_price
            })
        
        except Exception as e:
            print(f"[❌] Failed for {ticker}: {e}")

    df_log = pd.DataFrame(log_records)
    
    # Append to CSV log
    log_file = "reports/key_level_hit_log.csv"
    df_log.to_csv(log_file, mode="a", index=False, header=not os.path.exists(log_file))
    
    print(f"[💾] Key Level Hit Log saved to: {log_file}")
    print(df_log.to_string(index=False))


if __name__ == "__main__":
    alert_df, key_levels_df = generate_alert_report()
    log_key_level_hits(alert_df, key_levels_df)



[→] Checking USDCAD=X...
YF.download() has changed argument auto_adjust default to True
[→] Checking USDGBP=X...
[→] Checking USDNOK=X...
[→] Checking USDPLN=X...
[→] Checking USDAUD=X...
[→] Checking USDSGD=X...
[→] Checking USDJPY=X...
[→] Checking USDZAR=X...
[→] Checking USDBRL=X...
[→] Checking EURUSD=X...
[→] Checking EURGBP=X...
[→] Checking EURCHF=X...
[→] Checking EURPLN=X...
[→] Checking EURCZK=X...
[→] Checking EURNZD=X...
[→] Checking EURSEK=X...
[→] Checking EURZAR=X...
[→] Checking EURSGD=X...
[→] Checking GBPNOK=X...
[→] Checking GBPJPY=X...
[→] Checking GBPAUD=X...
[→] Checking GBPCAD=X...
[→] Checking SEKNOK=X...
[❌] Failed for SEKNOK=X: can only concatenate str (not "float") to str
[→] Checking SEKJPY=X...
[→] Checking CHFNOK=X...
[→] Checking CADNOK=X...
[→] Checking AUDNZD=X...
[→] Checking AUDJPY=X...
[→] Checking AUDSEK=X...
[→] Checking AUDCAD=X...
[→] Checking NZDSGD=X...
[→] Checking NZDCHF=X...
[→] Checking NZDNOK=X...
[→] Checking SGDJPY=X...
[→] Checking SGD

In [ ]:
print(ZONE_DEFINITIONS)
